In [2]:
!nvidia-smi

Mon Mar 11 22:54:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:17:00.0 Off |                  Off |
| 30%   59C    P0              86W / 300W |      4MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub==0.25.1
!pip install langchain==0.1.9

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ido7ukoc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ido7ukoc
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━

In [17]:
import os
import torch
from pydub import AudioSegment
import whisper
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:1


In [11]:
folder_path = os.getcwd()

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp4"):
        video_path = os.path.join(folder_path, file_name)

        # Extract audio from the video
        video = AudioSegment.from_file(video_path, format="mp4")
        
        # Create a corresponding mp3 file name
        audio_output_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}.mp3")

        # Export audio to mp3
        video.export(audio_output_path, format="mp3")

        print(f"Audio exported for {file_name} to {audio_output_path}")

Audio exported for Kint.mp4 to /home/jovyan/Kint.mp3
Audio exported for Kint2.mp4 to /home/jovyan/Kint2.mp3
Audio exported for Kint3.mp4 to /home/jovyan/Kint3.mp3


In [18]:
model = whisper.load_model("large")

In [19]:
result = ""

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp3"):
        audio_path = os.path.join(folder_path, file_name)

        # Transcribe the audio
        transcription_result = model.transcribe(audio_path)

        # Append the result to the string
        result += transcription_result["text"] + "\n"

# Now, 'result' contains the concatenated transcripts of all mp3 files in the folder
print(result)

 OK, good morning, everybody. So last week, we stopped at the question, how do we calculate the gradients for more complex loss functions and classifiers? And we saw that the thing that we need is the backpropagation algorithm. So we want to make use of the chain rule of calculus to determine the gradient of some complex compute graph. So we take one compute graph where we know the gradient for each of the nodes, and we use the chain rule to multiply those individual gradients, and thereby getting the gradient for the more complex function. And if, for example, we use a the compute graph for logistic regression, where we say, OK, we have two inputs, so two parameters for each of the input dimensions, and one bias term, our compute graph is this dot product over here, adding the bias term, getting this linear unit over here, taking the sigma. OK. And then we can use the sigma function of all of this and doing, performing, calculating our loss function. And for each of those steps, we kn

In [20]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

# Split the documents into chunks
chunks = text_splitter.split_text(result)
len(chunks)

203

In [21]:
import json
import os
from IPython.display import display, HTML

def save_json_to_file(data, filename):
    file_path = "chunks.json"
    # Check if the file already exists
    if os.path.exists(file_path):
        print("file already exists")
        # Load existing JSON data from the file
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    else:
        # If the file doesn't exist, initialize an empty list
        existing_data = []

    # Append the new array to the existing data
    existing_data.extend(chunks)

    # Convert the combined data to JSON format
    json_data = json.dumps(existing_data, indent=4)

    # Write the JSON data back to the file
    with open(file_path, "w") as json_file:
        json_file.write(json_data)

    print(f"JSON file has been updated at {file_path}")

def generate_download_link(filename):
    download_link = f'<a href="{filename}" download>Download json file containing chunks.</a>'
    display(HTML(download_link))

# Example usage:
data = chunks
filename = "chunks.json"
save_json_to_file(data, filename)
generate_download_link(filename)

JSON file has been updated at chunks.json
